In [1]:
%load_ext dotenv
%dotenv 
# UPSTAGE_API_KEY from https://console.upstage.ai/

In [2]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [3]:
%pip install langchain_faiss
%pip install gradio


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached gradio-5.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.4.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.12-py3-none-any.whl.metadata (1.9 kB)
  Using cached safehttpx-0.1.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
Using cached gradio-5.4.0-py3-none-any.whl (56.7 MB)
Using cached gradio_client-1.4.2-py3-none-any.whl (319 kB)
Using cached python_multipart-0.0.12-py3-none-any.whl (23 kB)
Using cached tomlkit-0.12.0-

In [10]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt

# 1. LLM 인스턴스 생성
llm = ChatUpstage()

# 3. 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    # PDF 문서 로드
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 4. 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    # 문서 임베딩 생성 및 저장
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    # 기존 인덱스가 있는지 확인
    if os.path.exists(faiss_index_path):
        # 기존 인덱스를 로드 (위험한 역직렬화 허용)
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        # 새로운 인덱스를 생성
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )
        # 인덱스를 디스크에 저장
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 5. 챗봇 프롬프트 및 체인 정의
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = chat_prompt | llm | StrOutputParser()

# 6. 챗봇 함수 정의
def chat(message, history):
    # 대화 히스토리 포맷팅
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    
    # 체인 호출
    response = chain.invoke({"message": message, "history": history_langchain_format})
    return response

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return fig

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하면, 맞춤형 투자 포트폴리오를 추천해드립니다.")

    # 사용자 입력: 투자 목표 및 리스크 범위
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 채팅창 및 결과 표시
    with gr.Row():
        chatbot = gr.ChatInterface(
            chat,
            examples=[
                "How to diversify my portfolio?",
                "What are the best investment strategies?",
                "Can you recommend a portfolio for retirement?",
            ],
            title="Invest-Portfolio Chatbot",
            description="투자 포트폴리오 추천 봇",
        )
        chatbot.chatbot.height = 300

    # 포트폴리오 추천 결과 및 시각화
    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    def recommend_portfolio(goal, risk, duration, history):
        # 사용자 입력을 바탕으로 챗봇과 대화하여 포트폴리오 추천
        message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. 이 조건에 맞는 포트폴리오를 추천해 주세요."
        recommendation = chat(message, history)
        
        # 임의의 포트폴리오 배분 결과 예시
        allocation = {
            "주식": 60 if risk == "높음" else 40,
            "채권": 30 if risk == "중간" else 40,
            "현금": 10 if risk == "낮음" else 20
        }
        
        # 포트폴리오 시각화 생성
        chart = display_portfolio_allocation(allocation)
        
        return recommendation, chart
    
    # 사용자 인터랙션 및 함수 호출
    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, chatbot],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    # 파일 경로
    file_path = "hana.pdf"
    
    # 문서 로딩 및 분할
    splits = load_and_split_docs(file_path)
    
    # FAISS 인덱싱 및 검색 준비
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    
    # Gradio 인터페이스 실행
    demo.launch()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2014, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1691, in preprocess_data
    raise InvalidComponentError(
gradio.exceptions.InvalidComponentError: <class 'gradio.chat_interface.ChatInterface'> Component not 

In [11]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt

# 1. LLM 인스턴스 생성
llm = ChatUpstage()

# 3. 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    # PDF 문서 로드
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 4. 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    # 문서 임베딩 생성 및 저장
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    # 기존 인덱스가 있는지 확인
    if os.path.exists(faiss_index_path):
        # 기존 인덱스를 로드 (위험한 역직렬화 허용)
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        # 새로운 인덱스를 생성
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )
        # 인덱스를 디스크에 저장
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 5. 챗봇 프롬프트 및 체인 정의
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = chat_prompt | llm | StrOutputParser()

# 6. 챗봇 함수 정의
def chat(message, history):
    # 대화 히스토리 포맷팅
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    
    # 체인 호출
    response = chain.invoke({"message": message, "history": history_langchain_format})
    return response

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return fig

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    # 사용자 입력: 투자 목표 및 리스크 범위
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 추가 사용자 메시지 입력 필드
    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    # 채팅창 및 결과 표시
    with gr.Row():
        chatbot = gr.ChatInterface(
            chat,
            examples=[
                "How to diversify my portfolio?",
                "What are the best investment strategies?",
                "Can you recommend a portfolio for retirement?",
            ],
            title="Invest-Portfolio Chatbot",
            description="투자 포트폴리오 추천 봇",
        )
        chatbot.chatbot.height = 300

    # 포트폴리오 추천 결과 및 시각화
    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    def recommend_portfolio(goal, risk, duration, message, history):
        # 사용자 입력을 바탕으로 챗봇과 대화하여 포트폴리오 추천
        combined_message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. {message}"
        recommendation = chat(combined_message, history)
        
        # 임의의 포트폴리오 배분 결과 예시
        allocation = {
            "주식": 60 if risk == "높음" else 40,
            "채권": 30 if risk == "중간" else 40,
            "현금": 10 if risk == "낮음" else 20
        }
        
        # 포트폴리오 시각화 생성
        chart = display_portfolio_allocation(allocation)
        
        return recommendation, chart
    
    # 사용자 인터랙션 및 함수 호출
    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message, chatbot],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    # 파일 경로
    file_path = "hana.pdf"
    
    # 문서 로딩 및 분할
    splits = load_and_split_docs(file_path)
    
    # FAISS 인덱싱 및 검색 준비
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    
    # Gradio 인터페이스 실행
    demo.launch()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2014, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1691, in preprocess_data
    raise InvalidComponentError(
gradio.exceptions.InvalidComponentError: <class 'gradio.chat_interface.ChatInterface'> Component not 

In [14]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt

# 1. LLM 인스턴스 생성
llm = ChatUpstage()

# 3. 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    # PDF 문서 로드
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 4. 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    # 문서 임베딩 생성 및 저장
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    # 기존 인덱스가 있는지 확인
    if os.path.exists(faiss_index_path):
        # 기존 인덱스를 로드 (위험한 역직렬화 허용)
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        # 새로운 인덱스를 생성
        vectorstore = FAISS.from_documents(
            documents=splits,
            embedding=embeddings
        )
        # 인덱스를 디스크에 저장
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 5. 챗봇 프롬프트 및 체인 정의
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = chat_prompt | llm | StrOutputParser()

# 6. 챗봇 함수 정의
def chat(message, history):
    # 대화 히스토리 포맷팅
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    
    # 체인 호출
    response = chain.invoke({"message": message, "history": history_langchain_format})
    return response

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return fig

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    # 사용자 입력: 투자 목표 및 리스크 범위
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 추가 사용자 메시지 입력 필드
    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    # 포트폴리오 추천 결과 및 시각화
    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    def recommend_portfolio(goal, risk, duration, message):
        # 빈 리스트로 history 초기화
        history = []
        
        # 사용자 입력을 바탕으로 챗봇과 대화하여 포트폴리오 추천
        combined_message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. {message}"
        recommendation = chat(combined_message, history)
        
        # 임의의 포트폴리오 배분 결과 예시
        allocation = {
            "주식": 60 if risk == "높음" else 40,
            "채권": 30 if risk == "중간" else 40,
            "현금": 10 if risk == "낮음" else 20
        }
        
        # 포트폴리오 시각화 생성
        chart = display_portfolio_allocation(allocation)
        
        return recommendation, chart
    
    # 사용자 인터랙션 및 함수 호출
    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    # 파일 경로
    file_path = "hana.pdf"
    
    # 문서 로딩 및 분할
    splits = load_and_split_docs(file_path)
    
    # FAISS 인덱싱 및 검색 준비
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    
    # Gradio 인터페이스 실행
    demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 49885 (\N{HANGUL SYLLABLE SIG}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 52292 (\N{HANGUL SYLLABLE CAE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 44428 (\N{HANGUL SYLLABLE GWEON}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/

In [15]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt

# 1. LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 3. 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 4. 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 5. 챗봇 프롬프트 및 체인 정의
def create_chat_chain():
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ])
    llm = create_llm()
    return chat_prompt | llm | StrOutputParser()

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    # 빈 리스트로 history 초기화
    history = []
    
    # LLM 인스턴스와 chain 생성
    chain = create_chat_chain()
    
    # 사용자 입력을 바탕으로 포트폴리오 추천 질문 생성
    combined_message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. 추가 메시지: {message}. 이 조건에 맞는 포트폴리오 배분과 설명을 해주세요."
    recommendation = chain.invoke({"message": combined_message, "history": history})

    # 배분 결과 예시를 LLM 응답으로부터 추출 (구체적인 예시는 LLM 응답 내용에 따라 다름)
    allocation = {
        "주식": 50,   # LLM 응답 내용 기반 예시
        "채권": 30,
        "현금": 20
    }

    # 시각화 생성
    chart = display_portfolio_allocation(allocation)

    # 배분 결과 설명 추가
    explanation = f"{recommendation}\n\n포트폴리오 배분 결과:\n주식: {allocation['주식']}%, 채권: {allocation['채권']}%, 현금: {allocation['현금']}%"

    return explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    # 사용자 입력: 투자 목표 및 리스크 범위
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 추가 사용자 메시지 입력 필드
    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    # 포트폴리오 추천 결과 및 시각화
    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    # 사용자 인터랙션 및 함수 호출
    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    # 파일 경로
    file_path = "hana.pdf"
    
    # 문서 로딩 및 분할
    splits = load_and_split_docs(file_path)
    
    # FAISS 인덱싱 및 검색 준비
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    
    # Gradio 인터페이스 실행
    demo.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [17]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt

# 1. LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 3. 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 4. 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 5. 챗봇 프롬프트 및 체인 정의
def create_chat_chain():
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ])
    llm = create_llm()
    return chat_prompt | llm | StrOutputParser()

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    # 빈 리스트로 history 초기화
    history = []
    
    # LLM 인스턴스와 chain 생성
    chain = create_chat_chain()
    
    # 사용자 입력을 바탕으로 포트폴리오 추천 질문 생성
    combined_message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. 추가 메시지: {message}. 이 조건에 맞는 포트폴리오 배분과 설명을 해주세요."
    recommendation = chain.invoke({"message": combined_message, "history": history})

    # 배분 결과 예시를 LLM 응답으로부터 추출 (구체적인 예시는 LLM 응답 내용에 따라 다름)
    allocation = {
        "주식": 50,   # LLM 응답 내용 기반 예시
        "채권": 30,
        "현금": 20
    }

    # 시각화 생성
    chart = display_portfolio_allocation(allocation)

    # 배분 결과 설명 추가
    explanation = f"{recommendation}\n\n포트폴리오 배분 결과:\n주식: {allocation['주식']}%, 채권: {allocation['채권']}%, 현금: {allocation['현금']}%"

    return explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    # 사용자 입력: 투자 목표 및 리스크 범위
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 추가 사용자 메시지 입력 필드
    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    # 포트폴리오 추천 결과 및 시각화
    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    # 사용자 인터랙션 및 함수 호출
    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    # 파일 경로
    file_path = "hana.pdf"
    
    # 문서 로딩 및 분할
    splits = load_and_split_docs(file_path)
    
    # FAISS 인덱싱 및 검색 준비
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    
    # Gradio 인터페이스 실행
    demo.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 49885 (\N{HANGUL SYLLABLE SIG}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 52292 (\N{HANGUL SYLLABLE CAE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 44428 (\N{HANGUL SYLLABLE GWEON}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/

In [18]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정 (예: 나눔 폰트 사용)
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')  # 나눔고딕 폰트 설정
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 챗봇 프롬프트 및 체인 정의
def create_chat_chain():
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ])
    llm = create_llm()
    return chat_prompt | llm | StrOutputParser()

# 포트폴리오 시각화 함수 (한글 폰트 적용 여부에 따라 레이블 선택)
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    history = []
    chain = create_chat_chain()
    
    combined_message = f"투자 목표는 '{goal}'이며, 리스크 허용 범위는 '{risk}'이고, 투자 기간은 {duration}년입니다. 추가 메시지: {message}. 이 조건에 맞는 포트폴리오 배분과 설명을 해주세요."
    recommendation = chain.invoke({"message": combined_message, "history": history})

    # LLM 응답에 포함된 배분 결과가 있을 경우만 표시
    allocation = {
        "Stocks": 60,
        "Bonds": 25,
        "Cash & Short-Term Bonds": 15
    }

    chart = display_portfolio_allocation(allocation)
    explanation = f"{recommendation}\n\n포트폴리오 배분 결과:\nStocks: {allocation['Stocks']}%, Bonds: {allocation['Bonds']}%, Cash & Short-Term Bonds: {allocation['Cash & Short-Term Bonds']}%"

    return explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.


In [21]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 챗봇 프롬프트 및 체인 정의
def create_chat_chain():
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ])
    llm = create_llm()
    return chat_prompt | llm | StrOutputParser()

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    # 빈 리스트로 history 초기화
    history = []
    
    # 포트폴리오 배분 결과 미리 정의
    allocation = {
        "Stocks": 60,
        "Bonds": 25,
        "Cash & Short-Term Bonds": 15
    }
    
    # LLM 설명 요청 메시지 생성 (한국어로 설명 요청)
    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"다음 포트폴리오 배분을 추천합니다: 주식 {allocation['Stocks']}%, 채권 {allocation['Bonds']}%, "
        f"현금 및 단기 채권 {allocation['Cash & Short-Term Bonds']}%. "
        f"이 포트폴리오 배분에 대해 한국어로 설명해 주세요. 추가 메시지: {message}"
    )
    
    # LLM으로 설명 생성
    explanation = chat(combined_message, history)
    
    # 설명 및 포트폴리오 배분 결과 표시
    full_explanation = (
        f"{explanation}\n\n포트폴리오 배분 결과:\n"
        f"주식: {allocation['Stocks']}%, 채권: {allocation['Bonds']}%, 현금/단기 채권: {allocation['Cash & Short-Term Bonds']}%"
    )

    # 포트폴리오 시각화
    chart = display_portfolio_allocation(allocation)
    
    return full_explanation, chart
# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Fon

In [22]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 챗봇 프롬프트 및 체인 정의
def create_chat_chain():
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant specialized in investment portfolio recommendations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ])
    llm = create_llm()
    return chat_prompt | llm | StrOutputParser()

# 포트폴리오 배분 계산 함수
def calculate_allocation(risk, duration):
    # 기본 배분값
    allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}
    
    # 리스크와 투자 기간에 따라 배분 조정
    if risk == "높음":
        allocation["Stocks"] += 10
        allocation["Bonds"] -= 5
        allocation["Cash & Short-Term Bonds"] -= 5
    elif risk == "낮음":
        allocation["Stocks"] -= 10
        allocation["Bonds"] += 5
        allocation["Cash & Short-Term Bonds"] += 5

    # 투자 기간에 따른 조정
    if duration >= 10:
        allocation["Stocks"] += 5
        allocation["Cash & Short-Term Bonds"] -= 5
    elif duration <= 3:
        allocation["Stocks"] -= 5
        allocation["Cash & Short-Term Bonds"] += 5
    
    return allocation

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    # 빈 리스트로 history 초기화
    history = []
    
    # 투자 목표와 리스크 허용 범위, 투자 기간에 따라 포트폴리오 배분 계산
    allocation = calculate_allocation(risk, duration)
    
    # LLM 설명 요청 메시지 생성 (한국어로 설명 요청)
    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"다음 포트폴리오 배분을 추천합니다: 주식 {allocation['Stocks']}%, 채권 {allocation['Bonds']}%, "
        f"현금 및 단기 채권 {allocation['Cash & Short-Term Bonds']}%. "
        f"이 포트폴리오 배분에 대해 한국어로 설명해 주세요. 추가 메시지: {message}"
    )
    
    # LLM으로 설명 생성
    explanation = chat(combined_message, history)
    
    # 설명 및 포트폴리오 배분 결과 표시
    full_explanation = (
        f"{explanation}\n\n포트폴리오 배분 결과:\n"
        f"주식: {allocation['Stocks']}%, 채권: {allocation['Bonds']}%, 현금/단기 채권: {allocation['Cash & Short-Term Bonds']}%"
    )

    # 포트폴리오 시각화
    chart = display_portfolio_allocation(allocation)
    
    return full_explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.


In [34]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    # allocation이 비어 있거나 0인 경우 기본값 설정
    if not allocation or all(value == 0 for value in allocation.values()):
        allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}

    labels = ["Stocks", "Bonds", "Cash & Short-Term Bonds"]
    sizes = [allocation.get("Stocks", 0), allocation.get("Bonds", 0), allocation.get("Cash & Short-Term Bonds", 0)]

    # 파스텔톤 색상 지정
    pastel_colors = ["#AEC6CF", "#FFB347", "#FF6961"]

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=pastel_colors)
    ax.axis('equal')  # 원형 차트로 유지

    return fig

# 포트폴리오 추천 함수
# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    history = []

    # LLM에게 배분 요청 메시지 생성
    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"주식, 채권, 현금 및 단기 채권의 비율을 각각 퍼센트로만 반환해 주세요. "
        f"예: 주식: 50%, 채권: 30%, 현금/단기 채권: 20%. 이후 설명은 별도의 단락에 작성해 주세요."
        f"추가 메시지 '{message}'에 대해서도 목표, 리스크, 기간을 고려한 맞춤형 응답을 제공해 주세요."
    )
    
    # LLM으로부터 응답 받기
    full_response = chat(combined_message, history)
    
    # 포트폴리오 배분과 설명 분리
    allocation = {}
    explanation = ""
    parsing_allocation = True
    for line in full_response.split("\n"):
        if parsing_allocation and ":" in line and "%" in line:
            asset, percentage = line.split(":", 1)
            try:
                allocation[asset.strip()] = float(percentage.strip().replace("%", ""))
            except ValueError:
                explanation += line + "\n"
                parsing_allocation = False
        else:
            explanation += line + "\n"

    # 포트폴리오 비율이 정확히 추출되었는지 확인하고, 기본 값으로 설정
    if not allocation:
        allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}
    
    # 포트폴리오 시각화 생성
    chart = display_portfolio_allocation(allocation)
    
    # 전체 설명과 배분 결과 반환
    full_explanation = (
        f"포트폴리오 배분 결과:\n"
        f"주식: {allocation.get('Stocks', 0)}%, 채권: {allocation.get('Bonds', 0)}%, "
        f"현금/단기 채권: {allocation.get('Cash & Short-Term Bonds', 0)}%\n\n"
        f"{explanation.strip()}"
    )

    return full_explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/matplotlib/axes/_axes.py:3290: RuntimeWarning: invalid value encountered in divide
  x = x / sx
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/bloc

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


In [37]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 포트폴리오 시각화 함수
def display_portfolio_allocation(allocation):
    labels = allocation.keys()
    sizes = allocation.values()
    
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    
    return fig

# 포트폴리오 추천 함수
def recommend_portfolio(goal, risk, duration, message):
    history = []

    # LLM에게 배분 요청 메시지 생성
    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"주식, 채권, 현금 및 단기 채권의 비율을 각각 퍼센트로만 반환해 주세요. "
        f"예: 주식: 50%, 채권: 30%, 현금/단기 채권: 20%. 이후 설명은 별도의 단락에 작성해 주세요."
        f"추가 메시지 '{message}'에 대해서도 목표, 리스크, 기간을 고려한 맞춤형 응답을 제공해 주세요."
    )
    
    # LLM으로부터 응답 받기
    full_response = chat(combined_message, history)
    
    # 포트폴리오 배분과 설명 분리
    allocation = {}
    explanation = ""
    parsing_allocation = True
    for line in full_response.split("\n"):
        if parsing_allocation and ":" in line and "%" in line:
            asset, percentage = line.split(":", 1)
            try:
                allocation[asset.strip()] = float(percentage.strip().replace("%", ""))
            except ValueError:
                explanation += line + "\n"
                parsing_allocation = False
        else:
            explanation += line + "\n"

    # 포트폴리오 비율이 정확히 추출되었는지 확인하고, 기본 값으로 설정
    if not allocation:
        allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}
    
    # 포트폴리오 시각화 생성
    chart = display_portfolio_allocation(allocation)
    
    # 전체 설명과 배분 결과 반환
    full_explanation = (
        f"포트폴리오 배분 결과:\n"
        f"주식: {allocation.get('Stocks', 0)}%, 채권: {allocation.get('Bonds', 0)}%, "
        f"현금/단기 채권: {allocation.get('Cash & Short-Term Bonds', 0)}%\n\n"
        f"{explanation.strip()}"
    )

    return full_explanation, chart

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 투자 포트폴리오 추천 시스템")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 메시지를 남기면 맞춤형 투자 포트폴리오를 추천해드립니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")

    with gr.Row():
        portfolio_result = gr.Textbox(label="포트폴리오 추천 결과", lines=5, placeholder="여기에 추천 포트폴리오가 표시됩니다.")
        portfolio_chart = gr.Plot(label="포트폴리오 배분 차트")

    submit_btn = gr.Button("포트폴리오 추천 받기")
    submit_btn.click(
        recommend_portfolio,
        inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
        outputs=[portfolio_result, portfolio_chart]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 49885 (\N{HANGUL SYLLABLE SIG}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
findfont: Font family 'NanumGothic' not found.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/processing_utils.py:158: UserWarning: Glyph 52292 (\N{HANGUL SYLLABLE CAE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/Library